# Categories in haskell

## Definition of a category

A category $C$ consists of four constituents:

1. A set $\mathrm{Ob}(C)$, elements of which are called *objects of $C$*.
2. For every pair of objects $c, d \in Ob(C)$, a set $C(c,d)$, elements of which are called *morphisms from $c$ to $d$*, often denoted $f : c \to d$.
3. For every object $c$, a specified morphism $\mathrm{id}_c \in C(c,c)$ called the *identity morphism for $c$*.
4. For every three objects $b,c,d \in Ob(C)$, and morphisms $f: c \to d$ and $g : c \to d$, a specified morphism $f\,;\,g : b \to d$, called the *composite of $f$ and $g$* often denoted as $g \cdot f$.

These four constituents are subject to the following three constraints (the *category laws*):

1. **Left Unital**: for any $f: c \to d$, the equation $id_c\,;\,f = f$ holds.
2. **Right Unital**: for any $f: c \to d$, the equation $f\,;\,id_d = f$ holds.
3. **Associative**: for any $f:c_1 \to c_2, g:c_2 \to c_3, h: c_3 \to c_4$, the equation $(f\,;\,g)\,;\,h = f\,;\,(g\,;\,h)$ holds.

## The typeclass `Category`

We can implement the notion of category as a haskell typeclass, where:
- For a given category $C$, the type `obj` characterises the set $Obj(C)$.
- The type `mor` characterises the set of morphisms.

Notice that this implementation as a haskell typeclass does not *guarantee* that an instance of `Category` satisfies the category laws. In order to show this, one has to prove for any object `x` and morphism `f`:
- `if (dom f == x) then cmp (idy x) f == Just f` (left unital)
- `if (cod f == y) then cmp f (idy y) == Just f` (right unital)

And finally, for any morphisms `f,g,h`:
- `cmp (cmp f g) h == cmp f (cmp g h)` (associative)

In [1]:
{-# language FlexibleInstances, FunctionalDependencies #-}

class Category obj mor | mor -> obj where
    dom :: mor -> obj
    cod :: mor -> obj
    idy :: obj -> mor
    cmp :: mor -> mor -> Maybe mor

## Implementing the walking arrow category

As an exercise, we can can implement the "walking arrow" category, often called $\mathbf{2}$. The full definition of the category is given below:

 - $Ob(\mathbf{2}) = \{a,b\}$
 - $\mathbf{2}(a,b) = \{f\}$
 - $\mathbf{2}(a,a) = \{id_a\}$
 - $\mathbf{2}(b,b) = \{id_b\}$ 
 - $\mathbf{2}(b,a) = \emptyset$
 
 We can implement the walking arrow category in haskell -- the objects of the category are given by the haskell type `TwoObj`, and the morphisms are given by the haskell category `TwoArr`.

In [2]:
data TwoObj = One | Two
data TwoArr = IdOne | IdTwo | OneToTwo

instance Category TwoObj TwoArr where
    dom IdTwo = Two
    dom _ = One
    cod IdOne = One
    cod _ = Two
    idy One = IdOne
    idy Two = IdTwo
    cmp IdOne OneToTwo = Just OneToTwo
    cmp OneToTwo IdTwo = Just OneToTwo
    cmp IdOne IdOne = Just IdOne
    cmp IdTwo IdTwo = Just IdTwo
    cmp _ _ = Nothing

## Preorders

### Definition of a preorder

A category $C$ is a *preorder* iff for every two objects $a,b \in C$, there is at most one morphism $a \to b$.

### Preorder example

There is a preorder $P$, s.t., $\mathrm{Ob}(P) = \mathbb{N}_{\geq 1}$, and where:

$$P(a,b) := \{x \in \mathbb{N}\,|\,x * a = b\}$$

Now let's consider some properties of this category.
- The identity on $12 \in \mathrm{Ob}(P)$ is $1 \in \mathrm{Ob}(P)$. In fact, for every object in $P$, the identity is $1$.
- If $x: a \to b$ and $y: b \to c$ are morphisms, there is a morphism $x\,;\,y$ to serve as their composite. Concretely, if $a * n = b$, and $b * m = c$, then $a * n * m = c$. The composite of $n$ and $m$ is therefore just $n * m$.
- Let's imagine that the objects of $P$ weren't just positive integers, but rather just $\mathbb{N}$. $P$ should still be a preorder.

A practical application of the notion of preorder to haskell is typeclasses -- the category that has as its objects typeclasses of haskell, and its morphisms the superclass relationship, is a preorder.

## Full subcategories

Let $C$ be any category, where $D \subseteq \mathrm{Ob}(C)$. The *full subcategory of $C$ spanned by $D$*, denoted $C_{\mathrm{Ob = D}}$, is defined by:

$$\mathrm{Ob}(C_{\mathrm{Ob} = D}) := D$$

and 

$$C_{\mathrm{Ob} = D}(d_1,d_2) = C(d_1,d_2)$$

Using our typeclass `Category`, we can make an instance that defines the full subcategory of Hask spanned by the objects `Bool` and `Int`. The only way I could figure out how to do this in a straightforward way was to come up with two "proxy types" `HaskBool` and `HaskInt` to stand in for the objects of `Hask`.

In [55]:
data BoolIntOb = HaskBool | HaskInt

data BoolIntArr = BoolToBool (Bool -> Bool) | IntToInt (Int -> Int) | BoolToInt (Bool -> Int) | IntToBool (Int -> Bool)

instance Category BoolIntOb BoolIntArr where
    dom (BoolToBool _) = HaskBool
    dom (BoolToInt _) = HaskBool
    dom _ = HaskInt
    cod (BoolToBool _) = HaskBool
    cod (IntToBool _) = HaskBool
    cod _ = HaskInt
    idy HaskBool = BoolToBool id
    idy HaskInt = IntToInt id
    cmp (BoolToBool f) (BoolToBool g) = Just (BoolToBool (g . f))
    cmp (BoolToBool f) (BoolToInt g) = Just (BoolToInt (g . f))
    cmp (IntToInt f) (IntToInt g) = Just (IntToInt (g .f))
    cmp (BoolToInt f) (IntToInt g) = Just (BoolToInt (g . f))
    cmp (BoolToInt f) (IntToBool g) = Just (BoolToBool (g . f))
    cmp (IntToInt f) (IntToBool g) = Just (IntToBool (g . f))
    cmp _ _ = Nothing
    
:t cmp (IntToInt id) (IntToBool even) 

cmp (IntToInt id) (IntToBool even) :: Maybe BoolIntArr

: 